In [1]:
!nvidia-smi

Tue May 17 07:34:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q transformers

In [3]:
#transformers
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler 
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer
from transformers import BertModel,BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [4]:
df = pd.read_csv('/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/clean_train.csv', header=0)
df.head()

,text_clean,Sentiment
0,it s a confusing odd time for the shopping pub...,Negative
1,in 2019 d2c ecommerce sales reached 1428 billi...,Positive
2,chinese residents are paying exorbitant prices...,Negative
3,list of supermarkets grocery shops and vegetab...,Neutral
4,its there any wonder tesco and other supermark...,Positive


In [5]:
df_test = pd.read_csv('/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/clean_test.csv', header=0)
df_test.head()

,text_clean,Sentiment
0,we may not have any toilet paper in our house ...,Positive
1,really whats the downside of coronavirus for a...,Positive
2,hello everyone we made amp sell high quality m...,Extremely Positive
3,happy to report that i jumped on the panic sho...,Positive
4,just been to the supermarket why do all women ...,Neutral


# Sentiment column ananlysis

In [6]:
df['Sentiment'].value_counts()

Positive              11381
Negative               9889
Neutral                7560
Extremely Positive     6618
Extremely Negative     5475
Name: Sentiment, dtype: int64

In [7]:
df['Sentiment'] = df['Sentiment'].map({'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1,
                                       'Positive': 2, 'Extremely Positive': 2})

In [8]:
df_test['Sentiment'] = df_test['Sentiment'].map({'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1,
                                                 'Positive': 2, 'Extremely Positive': 2})

In [9]:
df['Sentiment'].value_counts()

2    17999
0    15364
1     7560
Name: Sentiment, dtype: int64

## Class Balanceing by RandomOverSampler

In [10]:
# ros = RandomOverSampler()
# train_x, train_y = ros.fit_resample(np.array(df['text_clean']).reshape(-1, 1), np.array(df['Sentiment']).reshape(-1, 1))
# train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns=['text_clean', 'Sentiment'])

In [11]:
train_os = df

In [12]:
train_os['Sentiment'].value_counts()

2    17999
0    15364
1     7560
Name: Sentiment, dtype: int64

## Train - Validation - Test split

In [13]:
train_texts = train_os['text_clean'].values.tolist()
train_labels = train_os['Sentiment'].values.tolist()

In [14]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, stratify=train_labels, random_state=42)

In [15]:
test_texts = df_test['text_clean'].values.tolist()
test_labels = df_test['Sentiment'].values.tolist()

In [16]:
print(f"TRAINING DATA: {len(train_texts)}\nVALIDATION DATA: {len(val_texts)}\nTESTING DATA: {len(test_texts)}" )

TRAINING DATA: 32738
VALIDATION DATA: 8185
TESTING DATA: 3787


# BERT Sentiment analysis

In [17]:
MAX_LEN=128

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [19]:
train_encodings = tokenizer(train_texts, max_length=MAX_LEN,
                            padding='max_length', truncation=True)
val_encodings = tokenizer(val_texts, max_length=MAX_LEN,
                            padding='max_length', truncation=True)
test_encodings = tokenizer(test_texts, max_length=MAX_LEN,
                            padding='max_length', truncation=True)

In [20]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
test_dataset = TweetDataset(test_encodings, test_labels)

# Fine-tuning with Trainer

In [21]:
# training_args = TrainingArguments(
#     output_dir='/content/results',
#     num_train_epochs=20,
#     per_device_train_batch_size=128,
#     per_device_eval_batch_size=128,
#     warmup_ratio=0.1,
#     weight_decay=0.01,
#     logging_dir='/content/logs',
#     logging_steps=200
# )

# bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
# bert_model

In [22]:
# trainer = Trainer(
#     model=bert_model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset
# )

# trainer.train()

# Fine-tuning with native PyTorch/TensorFlow

In [23]:
from torch.utils.data import DataLoader
from transformers import AdamW

bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
bert_model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [24]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [25]:
from tqdm.notebook import tqdm

def train_model(model, train_dataloader, val_dataloader, epochs=20, device='cuda'):
    model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=5e-5)

    for epoch_num in range(epochs):
        for batch in tqdm(train_dataloader):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()
        
        total_val_loss = 0
        total_val_acc = 0

        with torch.no_grad():
            for batch in tqdm(val_dataloader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

                loss = outputs.loss
                logits = outputs.logits

                total_val_loss += loss.item()
                total_val_acc += (logits.argmax(axis=1) == labels).sum().item()
        
        print("Validation loss: {}".format(total_val_loss / (128 * len(val_dataloader))))
        print("Validation acc: {}".format(total_val_acc / (128 * len(val_dataloader))))


train_model(bert_model, train_dataloader, val_dataloader, 20)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

Validation loss: 0.002736105774602038
Validation acc: 0.877197265625


  0%|          | 0/256 [00:00<?, ?it/s]

...

  0%|          | 0/64 [00:00<?, ?it/s]

Validation loss: 0.003975805455411319
Validation acc: 0.90283203125


  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

Validation loss: 0.004101759304830921
Validation acc: 0.9071044921875


# BERT result

In [26]:
def test_model(model, test_dataloader, device='cuda'):
    model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=5e-5)

    total_test_loss = 0
    total_test_acc = 0

    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss
            logits = outputs.logits

            total_test_loss += loss.item()
            total_test_acc += (logits.argmax(axis=1) == labels).sum().item()
    
    print("Test acc: {}".format(total_test_acc / (128 * len(test_dataloader))))


test_model(bert_model, test_dataloader)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/30 [00:00<?, ?it/s]

Test acc: 0.8760416666666667
